In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from scipy.interpolate import CubicSpline

from drdmannturb.calibration import CalibrationProblem
from drdmannturb.shared.common import MannEddyLifetime
from drdmannturb.data_generator import OnePointSpectraDataGenerator

from drdmannturb.shared.enums import EddyLifetimeType, PowerSpectraType, DataType
from drdmannturb.shared.parameters import NNParameters, ProblemParameters, LossParameters, PhysicalParameters

plt.style.use("bmh")

plt.rc("text", usetex=True)
plt.rc("font", family="serif")

In [11]:
spectra_file = "data/Spectra_interp.dat"


domain = torch.logspace(-1, 2, 20)

L = 70
GAMMA = 3.7
SIGMA = 0.04

Uref = 21
zref = 1

pb = CalibrationProblem(
    nn_params = NNParameters(
        activations = [nn.GELU(), nn.GELU(), nn.GELU()]
    ),
    prob_params = ProblemParameters(
        data_type = DataType.CUSTOM,
        tol=1e-9,
    ),
    loss_params = LossParameters(
        alpha_reg = 1e-5
    ),
    phys_params = PhysicalParameters(
        L=L,
        Gamma=GAMMA,
        sigma=SIGMA,
        domain=domain,
        Uref=Uref,
        zref=zref,
    ),
)

In [12]:
parameters = pb.parameters
parameters[:3] = [np.log(L), np.log(GAMMA), np.log(SIGMA)]
pb.parameters = parameters[:len(pb.parameters)]

CustomData = torch.tensor(np.genfromtxt(spectra_file, skip_header=1, delimiter=","))
f = CustomData[:, 0]
k1_data_pts = 2 * torch.pi * f / Uref

In [13]:
DataPoints = [(k1, 1) for k1 in k1_data_pts]
Data = OnePointSpectraDataGenerator(
    data_points=DataPoints, k1_data_points=k1_data_pts.data.numpy(),
).Data


In [ ]:
pb.calibrate(data=Data)